We estimate a simple model of $n$ independent Bernoulli draws, with probability $\alpha$. First, we load the packages we use:

In [ ]:
using ContinuousTransformations
using DiffWrappers
using DynamicHMC
using MCMCDiagnostics
using Parameters

Then define a structure to hold the data. For this model, the number of draws equal to $1$ is a sufficient statistic.

In [2]:
"""
Toy problem using a Bernoulli distribution.

We model `n` independent draws from a ``Bernoulli(α)`` distribution.
"""
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws =1 in the data."
    s::Int
end

BernoulliProblem

Then make the type callable with the parameters *as a vector*.

In [3]:
function (problem::BernoulliProblem)(θ)
    α, = θ                                                # extract the parameter
    @unpack n, s = problem                                # extract the data
    s * log(α) + (n-s) * log(1-α) # log likelihood
end

Recall that we need to 

1. transform from $\mathbb{R}$ to the valid parameter domain $[0,1]$ for more efficient sampline, and

2. calculate the derivatives for this transformed mapping.

The helper packages take care of this.

In [4]:
p = BernoulliProblem(100, 40)                             # original problem
pt = TransformLogLikelihood(p, bridge(ℝ, Segment(0, 1)))  # transform
pt∇ = ForwardGradientWrapper(pt, [0.0]);                  # AD using ForwardDiff.jl

Finally, we sample from the posterior.

In [ ]:
sample, NUTS_tuned = NUTS_init_tune_mcmc(Base.Random.GLOBAL_RNG, pt∇, 1, 1000);
posterior = map(get_transformation(pt) ∘ get_position, sample);

`sample` holds the sample (along with diagnostic information), while `NUTS_tuned` is the tuned sampler which would allow continuation of sampling. To get the posterior for $\alpha$, we need to use `get_position` and transform.

In [6]:
posterior_α = first.(posterior)
mean(posterior_α)

0.4032038812140597

In [9]:
ess_α = effective_sample_size(posterior_α)

325.5188487615162

In [8]:
NUTS_statistics(sample)         # NUTS-specific statistics

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.95, min/25%/median/75%/max: 0.49 0.93 0.98 1.0 1.0
  termination: AdjacentTurn => 38% DoubledTurn => 62%
  depth: 1 => 56% 2 => 37% 3 => 7%
